# instalación de paquetes y verificación

In [6]:
# comprobamos que la versión de python sea inferior a 3.8, ya que para usar tensorflow 1.15 se recomienda usar python
# 3.7 o inferior
import sys
from datetime import datetime
import string
from object_detection.utils import label_map_util

python_version=sys.version.split("(")[0]
if int(sys.version.split(".")[1])>7:
    print("you are using a python version higer than 3.7.x, please install python 3.7.x")
else:
    print("congrats, you are using {} which is compatible with this notebook".format(python_version))


congrats, you are using 3.7.9  which is compatible with this notebook


In [7]:
# si todo sale bien,el test debería correr y los resultados aparecerán debajo. Es normal si se salta alguno de los tests 
# que hay
!python3 models/research/object_detection/builders/model_builder_tf1_test.py


Running tests under Python 3.7.9: /home/bigdata/anaconda3/envs/tf1/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
/home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_featur

# Variables de entorno

In [12]:
training_dir = "training_bb"
images_dir  = "images_bb"
inference_graph_dir = "inference_graph_faster_rcnn_{}_{}".format(datetime.now().day, datetime.now().month)
label_map = label_map_util.load_labelmap(training_dir+"/label_map.pbtxt")
label_map_dict = label_map_util.get_label_map_dict(label_map)
n_classes=len(label_map_dict.keys())

In [13]:
!sed 's/n_classes/{n_classes}/' {training_dir}/faster_rcnn_inception_v2_coco.config >{training_dir}/temp.config
!sed -i 's/training_dir/{training_dir}/' {training_dir}/temp.config
!cat {training_dir}/temp.config

model {
  faster_rcnn {
    num_classes: 3
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    sec

# creación del dataset

In [43]:
# borramos las carpetas de train test y validación si es que existían de un entrenamiento anterior
!rm -rf {images_dir}/train {images_dir}/test {images_dir}/val

In [44]:
# vamos a dividir nuestras imágenes y anotaciones en splits de train, test y validación.
# los argumentos -vr y -tr indican respectivamente los porcentajes de datos reservados para validación y test.
# el argumento -f indica el formato de las imágenes
!python3 scripts/train_test_split.py -i {images_dir} -o {images_dir} -tr 0.1 -vr 0.15 -f JPG
!tree -d {images_dir}

succesfully created validation split using:  15.0 % of data
succesfully created test split using:  10.0 % of data
succesfully created train split using:  75.0 % of data
images_bb
├── test
├── train
└── val

3 directories


In [ ]:
# vamos a convertir estas fotos en y anotaciones en datasets de formato COCO. Tenemos que hacerlo para los sets de train
# test y validación. Tenemos que pasar como argumento archivo txt con los labels siguiendo el formato indiado en la
# documentación oficial de labelme https://github.com/wkentaro/labelme/tree/master/examples/instance_segmentation. Un ejemplo
# de cómo debe ser este archivo se muestra debajo
!python3 scripts/labelme2coco.py {images_dir}/train {images_dir}/train/coco --labels {training_dir}/labels.txt
!python3 scripts/labelme2coco.py {images_dir}/test {images_dir}/test/coco --labels {training_dir}/labels.txt
!python3 scripts/labelme2coco.py {images_dir}/val {images_dir}/val/coco --labels {training_dir}/labels.txt
!tree -d {images_dir}

Creating dataset: images_bb/train/coco
Generating dataset from: images_bb/train/G0021379.json
Generating dataset from: images_bb/train/G0061833.json
Generating dataset from: images_bb/train/G0051528.json
Generating dataset from: images_bb/train/G0011084.json
Generating dataset from: images_bb/train/G0061742.json
Generating dataset from: images_bb/train/G0051626.json
Generating dataset from: images_bb/train/G0021382.json
Generating dataset from: images_bb/train/G0061856.json
Generating dataset from: images_bb/train/G0051538.json
Generating dataset from: images_bb/train/G0011073.json
Generating dataset from: images_bb/train/G0041513.json
Generating dataset from: images_bb/train/G0061780.json
Generating dataset from: images_bb/train/G0051624.json
Generating dataset from: images_bb/train/G0031397.json
Traceback (most recent call last):
  File "scripts/labelme2coco.py", line 187, in <module>
    main()
  File "scripts/labelme2coco.py", line 165, in main
    for (cnm, gid), msk in masks.item

<img src="labels_txt.PNG">

In [40]:
# ahora borramos las imágenes y anotaciones de las carpetas train, test y val, ya que la carpeta coco dentro de cada 
# una de estas carpetas ya contiene las imágenes originales
!rm {images_dir}/test/*
!mv {images_dir}/test/coco/* {images_dir}/test
!rm -rf {images_dir}/test/coco 

!rm {images_dir}/train/*
!mv {images_dir}/train/coco/* {images_dir}/train
!rm -rf {images_dir}/train/coco 

!rm {images_dir}/val/*
!mv {images_dir}/val/coco/* {images_dir}/val
!rm -rf {images_dir}/val/coco 

!tree -d {images_dir}

rm: cannot remove 'images_bb/test/coco': Is a directory
rm: cannot remove 'images_bb/train/coco': Is a directory
rm: cannot remove 'images_bb/val/coco': Is a directory
images_bb
├── test
│   ├── JPEGImages
│   └── Visualization
├── train
│   ├── JPEGImages
│   └── Visualization
└── val
    ├── JPEGImages
    └── Visualization

9 directories


In [40]:
# ahora tenemos que generar los tfrecords usando estos dataset en formato coco
!python3 models/research/object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
--train_image_dir={images_dir}/train \
--val_image_dir={images_dir}/val \
--test_image_dir={images_dir}/test \
--train_annotations_file={images_dir}/train/annotations.json \
--val_annotations_file={images_dir}/val/annotations.json \
--testdev_annotations_file={images_dir}/test/annotations.json \
--output_dir={training_dir}/tfrecord \
--include_masks=True

I1222 10:36:28.936357 140463654238016 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I1222 10:36:28.936712 140463654238016 create_coco_tf_record.py:412] 0 images are missing annotations.
I1222 10:36:28.936863 140463654238016 create_coco_tf_record.py:441] On image 0 of 241
I1222 10:36:44.573269 140463654238016 create_coco_tf_record.py:441] On image 100 of 241
I1222 10:37:01.575460 140463654238016 create_coco_tf_record.py:441] On image 200 of 241
I1222 10:37:08.215462 140463654238016 create_coco_tf_record.py:466] Finished writing, skipped 0 annotations.
I1222 10:37:08.221276 140463654238016 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I1222 10:37:08.221418 140463654238016 create_coco_tf_record.py:412] 0 images are missing annotations.
I1222 10:37:08.221460 140463654238016 create_coco_tf_record.py:441] On image 0 of 48
I1222 10:37:15.846401 140463654238016 create_coco_tf_record.py:466] Finished writing, 

# Entrenamiento y exportación del modelo

In [20]:
# entrenamos el modelo usando estos tfrecords. El modelo resultante se guardará el carpeta training_bb
!python3 models/research/object_detection/model_main.py --alsologtostderr \
--model_dir={training_dir}/ \
--pipeline_config_path={training_dir}/faster_rcnn_inception_v2_coco.config \
--num_train_steps=1000000

W1030 11:44:48.030372 140699655509824 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 1000000
I1030 11:44:48.030524 140699655509824 config_util.py:552] Maybe overwriting train_steps: 1000000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1030 11:44:48.030573 140699655509824 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I1030 11:44:48.030612 140699655509824 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1030 11:44:48.030652 140699655509824 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1030 11:44:48.030704 140699655509824 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, expor

# para correr el entrenamiento en la consola, copiar y pegar este comando

training_dir=training_bb
python3 models/research/object_detection/model_main.py --alsologtostderr \
--model_dir=$training_dir/ \
--pipeline_config_path=$training_dir/faster_rcnn_inception_v2_coco.config \
--num_train_steps=1000000

In [21]:
# aquí nos vamos a fijar en los archivos model.ckpt y vamos a buscar el que tenga el número más alto, ya que este número
# indica el step del proceso de entrenamiento en el que se realizó el checkpoint del modelo
!dir {training_dir}

checkpoint
eval_0
events.out.tfevents.1604054901.bigdata-alineware
events.out.tfevents.1604058311.bigdata-alineware
faster_rcnn_inception_v2_coco.config
graph.pbtxt
label_map.pbtxt
model.ckpt-1051.data-00000-of-00001
model.ckpt-1051.index
model.ckpt-1051.meta
model.ckpt-1317.data-00000-of-00001
model.ckpt-1317.index
model.ckpt-1317.meta
model.ckpt-1571.data-00000-of-00001
model.ckpt-1571.index
model.ckpt-1571.meta
model.ckpt-1839.data-00000-of-00001
model.ckpt-1839.index
model.ckpt-1839.meta
model.ckpt-785.data-00000-of-00001
model.ckpt-785.index
model.ckpt-785.meta
test.record
train.record
val.record


In [22]:
# por si no lo ves claro, corriendo esto te dirá qué debes poner en el script siguiente que generará el grafo de inferencia
import glob
import os
max_step=0
for file in glob.glob(training_dir+"/*"):
    if ("model.ckpt" in  file):
        step=int (file.split(os.sep)[1].split(".")[1].split("-")[1])
        max_step= step if step>max_step else max_step
        model_checkpoint_prefix="model.ckpt-{}".format(max_step)
print("En el campo trained_checkpoint debes poner lo siguiente: \n{}/model.ckpt-".format(training_dir)+str(max_step))

En el campo trained_checkpoint debes poner lo siguiente: 
training_bb/model.ckpt-1839


In [23]:
# ahora vamos a exportar este modelo como un grafo de inferencia. Este grafo se podrá usar para generar predicciones luego.
# Debemos sustituir los parámetros de entrada ["pipeline_config_path","trained_checkpoint_prefix","output_directory"]
# por los nuestros. Es recomendable elegir un nombre reconocible para la carpeta de salida, como por ejemplo
# "inference_graph" seguido del modelo entrenado y  la fecha y la hora
!python3 models/research/object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path training_bb/faster_rcnn_inception_v2_coco.config \
--trained_checkpoint_prefix {training_dir}/{model_checkpoint_prefix} \
--output_directory {inference_graph_dir}

Instructions for updating:
Please use `layer.__call__` method instead.
W1030 12:06:53.835841 140651862783808 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I1030 12:06:54.633029 140651862783808 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I1030 12:06:54.721038 140651862783808 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
I1030 12:06:54.721306 140651862783808 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1030 12:06:54.758372 140

In [25]:
!python3 scripts/tf_text_graph_faster_rcnn.py \
--input {inference_graph_dir}/frozen_inference_graph.pb \
--config {training_dir}/faster_rcnn_inception_v2_coco.config \
--output {inference_graph_dir}/graph.pbtxt

Number of classes: 3
Scales:            [0.25, 0.5, 1.0, 2.0]
Aspect ratios:     [0.5, 1.0, 2.0]
Width stride:      16.000000
Height stride:     16.000000
Features stride:   16.000000


# Pediciendo en imágenes

In [ ]:
# debemos sustituir el parámetro -m por la carpeta donde está nuestro modelo exportado 
!python3 scripts/predict_on_images.py \
-i images_bb/test \
-o predictions_image_bb \
-m {inference_graph_dir} \
-l {training_dir}/label_map.pbtxt \
-f JPG